In [1]:
import unittest
import h5py

#git repo specific setup
import sys
sys.path.append('../utils')

from training_utils import *
from evaluation import *

In [2]:
config = BertConfig.from_pretrained("bert-large-uncased", output_hidden_states=True)
model = PretrainedBertSquad2Faster('../../bert_base_squad_1e-5_adam_4batchsize_2epochs_weights_BERT_ONLY.h5')

In [8]:
import unittest
import h5py

#git repo specific setup
import sys
sys.path.append('../utils')

from training_utils import *
from evaluation import *

class TestBERTImageGenerator(unittest.TestCase):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = PretrainedBertSquad2Faster('../../bert_base_squad_1e-5_adam_4batchsize_2epochs_weights_BERT_ONLY.h5').model

        train = h5py.File('../../../SQuADv2/train_386.h5', 'r')
        self.start_ids = train['input_start']
        self.end_ids = train['input_end']
        self.train_inputs = np.array(train['input_ids'])
        self.train_masks = np.array(train['attention_mask'])
        self.labels = np.vstack([self.start_ids, self.end_ids]).T
        self.offset = 2
        self.start_idx = 14325
        self.end_idx = 17895

        self.images = BERTImageGenerator('../../../data/train/',
                            self.labels,
                            batch_size=1,
                            start_idx = self.start_idx,
                            end_idx = self.end_idx)

        self.subset = self.images[self.offset]

    def setUp(self):
        self.idx = self.start_idx + self.offset

    def test_embeddings_shape(self):
        #extract image 2 with starting index 14325, leading to image 14325

        self.assertEqual(len(self.subset), 2)
        self.assertEqual(self.subset[0].shape, (1, 24, 386, 1024))
        self.assertEqual(self.subset[1].shape, (1, 2))

    def test_offset(self):

        manual_load = h5py.File('../../../data/train/%d.h5' %self.idx, 'r')
        self.assertTrue((np.array(manual_load['hidden_state_activations'])[0][-1] == self.subset[0][0][0][-1]).all())

    def test_label_ids(self):
        self.assertEqual(self.subset[1][0][0], self.start_ids[self.idx])
        self.assertEqual(self.subset[1][0][1], self.end_ids[self.idx])

    def test_embeddings_with_model(self):

        embeddings, outputs = self.model.predict([self.train_inputs[[self.idx]], self.train_masks[[self.idx]]])
        non_zero_idx = np.sum(self.train_inputs[[self.idx]] != 0)
        self.assertTrue((embeddings[12][0][:non_zero_idx][-1] == self.subset[0][0][12][-1]).all())

    def test_shuffle_ids(self):
        pass

    def test_shuffled_labels(self):
        pass

#if __name__ == '__main__':
#    unittest.main()


In [2]:
train = h5py.File('../../../SQuADv2/train_386.h5', 'r')
train_inputs = np.array(train['input_ids'])
train_masks = np.array(train['attention_mask'])
start_ids = train['input_start']
end_ids = train['input_end']
labels = np.vstack([start_ids, end_ids]).T
        
images = BERTImageGenerator('../../../data/train/', 
                            labels, 
                            batch_size=1, 
                            start_idx = 14325, 
                            end_idx = 17895,
                            shuffle = False)

In [3]:
p = images[2]

In [65]:
p[0].shape == (1, 24, 386, 1024)

True

In [9]:
a  = TestBERTImageGenerator()

In [11]:
a

array([[19., 24.]])

In [5]:
images.subset

AttributeError: 'BERTImageGenerator' object has no attribute 'subset'

In [29]:
a = h5py.File('../../../data/train/14327.h5', 'r')

In [70]:
np.array(a['hidden_state_activations'])[0][-1]

array([-0.4621777 ,  0.27707544, -0.20735587, ..., -0.00903864,
        0.2197759 ,  0.00766955], dtype=float32)

In [71]:
p[0][0][0][-1]

array([-0.46217769,  0.27707544, -0.20735587, ..., -0.00903864,
        0.2197759 ,  0.00766955])

In [30]:
np.array(a['hidden_state_activations']).shape

(25, 275, 1024)

In [100]:
start_ids[14327], end_ids[14327]

(231.0, 232.0)

In [101]:
p = np.array(a['hidden_state_activations'])

In [102]:
p.shape

(25, 275, 1024)

In [32]:
p[1]

array([[231., 232.]])

In [33]:
p[0].shape

(1, 24, 386, 1024)

In [35]:
p[0][0][0][-1]

array([-0.46217769,  0.27707544, -0.20735587, ..., -0.00903864,
        0.2197759 ,  0.00766955])

In [83]:
embeddings, outputs = model.model.predict([train_inputs[[14327]], train_masks[[14327]]])

In [84]:
outputs[0][-1]

array([ 0.11816142, -0.38899085,  0.632986  , ...,  0.30848777,
       -1.1854073 , -0.44034505], dtype=float32)

In [85]:
np.array(a['sequence_outputs']).shape

(1, 275, 1024)

In [86]:
idx = np.sum(train_inputs[[14327]] != 0)

In [112]:
idx

275

In [89]:
embeddings[12][:idx].shape

(1, 386, 1024)

In [106]:
len(embeddings)

25

In [120]:
embeddings[12][0][:idx][-1] == p[0][0][12][-1]

array([ True,  True,  True, ...,  True,  True,  True])

In [99]:
p[0][0][12][-1]

array([ 0.3705852 ,  0.40124345, -0.39207998, ...,  0.58259469,
       -0.31270698,  0.32212216])

In [110]:
p[0][0][0][-1]

array([-0.46217769,  0.27707544, -0.20735587, ..., -0.00903864,
        0.2197759 ,  0.00766955])